In [40]:
!pip install selenium
!pip install beautifulsoup4
!pip install bs4
!pip install requests

In [41]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [42]:
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

In [43]:
# Import libraries
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import time
import datetime
import pandas as pd
import requests

In [44]:
def scraper():
  # Webdriver setting
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',options=chrome_options)

  # Get website url
  url = 'https://www.airbnb.com/'
  driver.get(url)
  return driver


In [45]:
class pageScrape:

    # All information in elements of listings
    LISTING_INFORMATION = {'url': {'tag': 'a', 'get': 'href'},
                           'name': {'tag': 'span',  'class' : '_1whrsux9'},
                           'superhost': {'tag': 'div', 'class': '_ufoy4t'},
                           'desc': {'tag': 'div', 'class': '_b14dlit'},
                           'rooms': {'tag': 'div', 'class': '_kqh46o'},
                           'facilities': {'tag': 'div', 'class': '_kqh46o', 'order': 1},
                           'rating_reviews': {'tag': 'span', 'class': '_18khxk1'},
                           'price': {'tag': 'span', 'class': '_155sga30'}
                           }
    # Constructor
    def __init__(self, driver, curr_url):
        self.driver = driver
        self.curr_url = curr_url


    def using_selenium(self):
        ''''
        Get listings elements from search url
        using selenium
        '''
        # Getting page source/html
        #driver = self.driver
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        #listings = soup.find_all('div', {'class' : '_8s3ctt'})
        # Getting single listings element
        #listing = soup.find_all('div', '_gig1e7')
        return soup

    # Get all elements with target information
    def using_requests(self):
        ''''
        Get listings elements from searched url using requests
        '''
        soup = BeautifulSoup(requests.get(self.curr_url).content, 'html.parser')
        listings = soup.find_all('div', class_ = '_8s3ctt')
        print(len(listings))
        return listings

    def extract_single_listings(self, listings,loc):
        '''
        Extract all listings in single page
        '''

        listings_list = list()

        for listing in listings:
            listings_dict = dict()
            listings_dict['city'] = loc
            # Iterating for each information
            for info in pageScrape.LISTING_INFORMATION:
                try:
                    if 'class' in pageScrape.LISTING_INFORMATION[info]:
                        element = listing.find_all(pageScrape.LISTING_INFORMATION[info]['tag'], pageScrape.LISTING_INFORMATION[info]['class'])
                    elif 'span' in pageScrape.LISTING_INFORMATION[info]:
                        element = listing.find_all(pageScrape.LISTING_INFORMATION[info]['tag'], pageScrape.LISTING_INFORMATION[info]['span'])
                    else:
                        element = listing.find_all(pageScrape.LISTING_INFORMATION[info]['tag'])


                    # Element
                    orders = pageScrape.LISTING_INFORMATION[info].get('order', 0)
                    elements = element[orders]

                    # Values
                    if 'get' in pageScrape.LISTING_INFORMATION[info]:
                        output = elements.get(pageScrape.LISTING_INFORMATION[info]['get'])
                    else:
                        output = elements.get_text()

                    listings_dict[info] = output
                except:
                    listings_dict[info] = 'empty'


            listings_list.append(listings_dict)

        return listings_list

In [62]:
def search_and_go(loc, driver):
    ''''
    Melakukan pencarian kota berdasarkan input
    loc = input kota
    '''
    # Menentukan lokasi search box
    #search = driver.find_element_by_class_name('_1xq16jy')
    #search = driver.find_element_by_xpath("//input[@placeholder='Where are you going?']")
    time.sleep(3)
    search = WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, "//input[@placeholder='Where are you going?']")))
    #search = WebDriverWait(driver, 20).until(expected_conditions.presence_of_element_located((By.CLASS_NAME, "_1xq16jy")))

    #driver.find_element_by_class_name('_1xq16jy').click()
    driver.implicitly_wait(5)
    search.send_keys(loc)
    driver.implicitly_wait(10)
    time.sleep(2)
    #actions = ActionChains(driver)
    #search.send_keys(Keys.TAB + Keys.TAB + Keys.TAB + Keys.TAB + Keys.TAB + Keys.ENTER)
    WebDriverWait(driver, 20).until(expected_conditions.presence_of_element_located((By.CLASS_NAME, "_1mzhry13"))).click
    # Send keys without element
    
    #search.send_keys(Keys.RETURN)
    #actions.send_keys(Keys.TAB)
    #time.sleep(5)
    #actions.send_keys(Keys.TAB)
    #time.sleep(5)
    #actions.send_keys(Keys.TAB)
    #time.sleep(5)
    #actions.send_keys(Keys.TAB)
    #time.sleep(5)
    #actions.send_keys(Keys.TAB)
    #time.sleep(5)
    #actions.send_keys(Keys.RETURN)

    go = WebDriverWait(driver, 30).until(expected_conditions.element_to_be_clickable((By.CLASS_NAME, '_1mzhry13'))).click()
    curr_url = driver.current_url
    return curr_url


    # find
    #go = driver.find_element_by_class_name('_m9v25n')
    #return go.click()

def iterating_pages(driver, loc):
    ''''
    Scrape all listings in all page
    '''
    scraped_data = list()

    while True:
        try:
            # Process
            print('Processing city: {}'.format(loc))
            current_page = driver.current_url
            #print(current_page)
            soup = pageScrape(driver=driver, curr_url=current_page)
            page = soup.using_requests()
            if len(page) > 0:
              page_list = soup.extract_single_listings(listings = page, loc=loc)
              scraped_data.append(page_list)

              # Move to next page
              time.sleep(2)
              next_page = WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.CLASS_NAME, '_za9j7e')))
              next_page.click()
            else:
              break
        except:
            #print('Keknya ada yang error')
            print('No more page left. Scraping is done')
            driver.close()
            break

    return scraped_data


In [60]:
def to_pandas(var):
    ''''
    Convert scraped data to dataframe and save as csv
    var = list of scraped data
    '''
    dataframe = pd.DataFrame()
    for dim in var:
      df_dic = pd.DataFrame.from_records(dim)
      dataframe = dataframe.append(df_dic, ignore_index=True)
    dataframe = dataframe.drop_duplicates()
    return dataframe

def to_csv(loc, dataframe):
    '''
    Export to csv based on time exporting
    '''
    path = '/content/drive/MyDrive/Airbnb/exported_dataset'
    #path_ubuntu = '/home/expiatio/PycharmProjects/AirBnB/exported_dataset'
    ts = time.time()
    timenow = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d%H%M%S_%f')

    return dataframe.to_csv(os.path.join(path, timenow+'-'+loc+'.csv'),
                            index=False, sep=';')

In [ ]:
while True:
    print('Menu')
    print('1. Scrape City')
    print('2. Exit\n')
    menu = input('Choose a menu:')

    if menu == str(1):
      locs = list(map(str, input('Please input city: ').strip().split(',')))
      print(locs)
      for loc in locs:
        print(loc)
        driver = scraper()
        # search = driver.find_element_by_class_name('_1xq16jy').click()
        curr_url = search_and_go(loc=loc, driver=driver)

        test_pages = iterating_pages(driver=driver, loc=loc)
        test_scrape = to_csv(loc=loc, dataframe=to_pandas(test_pages))
    elif menu == str(2):
        break
    else:
        print('I know you wanna go home')
        break


Menu
1. Scrape City
2. Exit

Choose a menu:1
Please input city: bangkalan,banyuwangi,blitar, bojonegoro,bondowoso,gresik,jember,kediri,lamongan,lumajang,madiun,magetan,malang,mojokerto,nganjuk,ngawi,pacitan,pamekasan,pasuruan,ponorogo,probolinggo,sampang,sidoarjo,situbondo,sumenep,trenggalek,tuban,tulungagung,kota batu,surabaya
['bangkalan', 'banyuwangi', 'blitar', ' bojonegoro', 'bondowoso', 'gresik', 'jember', 'kediri', 'lamongan', 'lumajang', 'madiun', 'magetan', 'malang', 'mojokerto', 'nganjuk', 'ngawi', 'pacitan', 'pamekasan', 'pasuruan', 'ponorogo', 'probolinggo', 'sampang', 'sidoarjo', 'situbondo', 'sumenep', 'trenggalek', 'tuban', 'tulungagung', 'kota batu', 'surabaya']
bangkalan
Processing city: bangkalan
8
No more page left. Scraping is done
banyuwangi
Processing city: banyuwangi
25
Processing city: banyuwangi
30
Processing city: banyuwangi
20
Processing city: banyuwangi
20
Processing city: banyuwangi
28
No more page left. Scraping is done
blitar
Processing city: blitar
30
Pr

bangkalan,banyuwangi,blitar,bojonegoro,bondowoso,gresik,jember,kediri,lamongan,lumajang,madiun,magetan,malang,mojokerto,nganjuk,ngawi,pacitan,pamekasan,pasuruan,ponorogo,probolinggo,sampang,sidoarjo,situbondo,sumenep,trenggalek,tuban,tulungagung,kota batu,surabaya